In [ ]:
def reproduceResult():
  seed_value= 0

  
  with tf.device("/cpu:0"):
    ...


  os.environ['PYTHONHASHSEED']=str(seed_value)
  np.random.seed(0)
  rn.seed(0)


  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                                          inter_op_parallelism_threads=1)


  tf.compat.v1.set_random_seed(seed_value)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  tf.compat.v1.keras.backend.clear_session()

  


In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import os 
import numpy as np
import tensorflow as tf
import random as rn
from tensorflow import keras

reproduceResult()
import tempfile
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K


from tensorflow.keras import callbacks

from keras_lr_finder import LRFinder
from clr_callback import CyclicLR

import tensorflow_model_optimization as tfmot



import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [ ]:
temp = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Dataset/merged_training.pkl')

In [ ]:
temp

,text,emotions
27383,i feel awful about it too because it s my job ...,sadness
110083,im alone i feel awful,sadness
140764,ive probably mentioned this before but i reall...,joy
100071,i was feeling a little low few days back,sadness
2837,i beleive that i am much more sensitive to oth...,love
...,...,...
566,that was what i felt when i was finally accept...,joy
36236,i take every day as it comes i m just focussin...,fear
76229,i just suddenly feel that everything was fake,sadness
131640,im feeling more eager than ever to claw back w...,joy


In [ ]:
data_90, data_002= train_test_split(temp, test_size=0.02, stratify = temp['emotions'], random_state = 42)

In [ ]:
data_002

,text,emotions
34509,i feel hopeless depressed angry,sadness
55118,i feel so vain why have you put me through so ...,sadness
40376,i feel like i am paying to be abused,sadness
29588,i feel accurately i have returned to this fait...,joy
12867,i feel confused and start the same task over,fear
...,...,...
64524,i feel so virtuous having made this for dinner...,joy
58585,i feel the single version was kind of messy an...,sadness
20443,i feel like all they watch are the obnoxious c...,anger
77681,i found a quote tonight that rather sums up ho...,joy


In [ ]:
data_002.emotions.value_counts()

joy         2822
sadness     2424
anger       1146
fear         954
love         691
surprise     300
Name: emotions, dtype: int64

In [ ]:
data_002.to_csv(r'/content/drive/MyDrive/Colab Notebooks/Dataset/IMDB/carer_0.02_percent.csv', index=True,header=True)

In [ ]:
temp = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/IMDB/carer_0.02_percent.csv')

del temp['Unnamed: 0']
temp


,text,emotions
0,i feel hopeless depressed angry,sadness
1,i feel so vain why have you put me through so ...,sadness
2,i feel like i am paying to be abused,sadness
3,i feel accurately i have returned to this fait...,joy
4,i feel confused and start the same task over,fear
...,...,...
8332,i feel so virtuous having made this for dinner...,joy
8333,i feel the single version was kind of messy an...,sadness
8334,i feel like all they watch are the obnoxious c...,anger
8335,i found a quote tonight that rather sums up ho...,joy


In [ ]:
temp.emotions.value_counts()

joy         2822
sadness     2424
anger       1146
fear         954
love         691
surprise     300
Name: emotions, dtype: int64

In [ ]:
max = 0
sum = 0
for sentence in temp['text']:
  val = len(list(sentence.split(" ")))
  sum +=val
  if max < val:
    max = val

max

94

In [ ]:
sum/4169

38.25353801870952

In [ ]:
# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(temp['text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

list_tokenized_train = tokenizer.texts_to_sequences(temp['text'])
  
  
MAX_LEN = 40
X_train = pad_sequences(list_tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train = temp['emotions']


X_train, X_test = train_test_split(X_train, test_size=0.2,  random_state = 42)
y_train, y_test = train_test_split(y_train, test_size=0.2,  random_state = 42)

encoding = {
    'joy': 0,
    'sadness': 1,
    'anger': 2,
    'fear': 3,
    'love': 4,
    'surprise': 5
}

y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]



y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(6669, 40)
(1668, 40)
(6669, 6)
(1668, 6)


In [ ]:
def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open('/content/drive/MyDrive/Colab Notebooks/Dataset/cc.en.300.vec') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)


In [ ]:
from tensorflow.keras import regularizers
import time
LOG_DIR = f"{int(time.time())}"
seed_value= 0



def build_model(hp):
  
  reproduceResult()

  print('Ya it comes here')
  unit_attention = hp.Int("attention_unit",min_value =32, max_value = 128, step = 16)
  fake_val = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_unit = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn_1_dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_unit = hp.Int("lstm_unit",min_value =64, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_unit = hp.Int("cnn_2_unit",min_value =64, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn_2_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)



  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,trainable=True,
                          weights = [embedd_matrix])(seq_input)


  cnn = keras.layers.Conv1D(cnn_1_unit,3)(embedded)
  cnn = keras.layers.Activation(activation='relu')(cnn)
  cnn = keras.layers.BatchNormalization()(cnn)
  cnn = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn)

  attention_vec = keras.layers.TimeDistributed(keras.layers.Dense(unit_attention))(cnn)
  attention_vec = keras.layers.Reshape((38,unit_attention))(attention_vec)
  attention_vec = keras.layers.Activation('relu', name = 'cnn_attention_vec')(attention_vec)
  attention_output = keras.layers.Dot(axes = 1)([cnn, attention_vec])


  lstm = keras.layers.GRU(lstm_unit,return_sequences=True,input_shape =(38,))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)
  
  

  cnn_2 = keras.layers.Conv1D(cnn_2_unit,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

  return model


stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')


clr_step_size = int((len(X_train)/16))
base_lr = 2e-5
max_lr = 1e-3
mode = 'exp_range'


clr = CyclicLR(base_lr = base_lr, max_lr = max_lr , step_size = clr_step_size, mode = mode)



tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 30,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train,y = y_train,epochs = 20, batch_size = 32,callbacks = [stop], 
             validation_data = (X_test,y_test))


tuner.results_summary()

Trial 13 Complete [00h 06m 46s]
val_accuracy: 0.8986810445785522

Best val_accuracy So Far: 0.8986810445785522
Total elapsed time: 02h 32m 01s

Search: Running Trial #14

Hyperparameter    |Value             |Best Value So Far 
attention_unit    |128               |48                
cnn_1_unit        |80                |32                
cnn_1_dropout     |0.2               |0.2               
lstm_unit         |192               |224               
lstm_dropout      |0.4               |0.2               
cnn_2_unit        |160               |96                
cnn_2_dropout     |0.2               |0.3               

Ya it comes here
Epoch 1/20
209/209 [==============================] - 60s 275ms/step - loss: 2.2553 - accuracy: 0.2624 - val_loss: 1.9523 - val_accuracy: 0.1085
Epoch 2/20
209/209 [==============================] - 58s 279ms/step - loss: 2.0559 - accuracy: 0.2809 - val_loss: 1.8012 - val_accuracy: 0.2740
Epoch 3/20
209/209 [==============================] - 58s 278ms/s

KeyboardInterrupt: ignored